In [2]:
import torch
from models import prepare_mnist, prepare_fashion_mnist, prepare_cifar10

import numpy as np
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from models import ModelConf
writer=SummaryWriter(f"CIFAR-10")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("device:", device)
conf = ModelConf(
    batch_size=128,
    n_hid=32*32,
    n_vis=32*32,
    n_epoch=10,
    dataset="cifar10",
    model_name="rbm",
    optimizer="adam"
)
conf.device = device

train_datasets, train_loader, test_datasets, test_loader = prepare_cifar10(batch_size=conf.batch_size)

device: cuda
Files already downloaded and verified
Files already downloaded and verified


In [3]:
from models import AutoEncoder, RBM, MLPAutoEncoder, VAE

def generate_vv(model, conf, fast=False):
    for epoch in tqdm(range(conf.n_epoch)):
        vv = np.zeros((len(train_datasets), conf.n_hid))
        if fast:
            for idx, (data, target) in enumerate(train_loader):
                batch_size = data.size(0)
                n_iter = len(train_loader) * epoch + idx
                recon_loss = model.train_step(data.to(conf.device)).to("cpu")
                writer.add_scalar(f"{model.__class__.__name__}/{str(conf)}/recon_loss", recon_loss, n_iter)
                vv[idx*batch_size:(idx+1)*batch_size, :] = model.encode(data.view(-1, conf.n_vis).to(conf.device)).detach().cpu().numpy()
        else:
            for idx, (data, target) in enumerate(train_loader):
                if conf.whitening_vis:
                    batch_size = data.size(0)
                    n_iter = len(train_loader) * epoch
                    batch_white = np.dot(data.view(batch_size, -1).numpy(), P)
                    batch_white = torch.from_numpy(batch_white.astype(np.float32)).clone().view(batch_size, -1)
                    vv[idx*batch_size:(idx+1)*batch_size, :] = model.encode(batch_white.to(conf.device)).detach().cpu().numpy()
                    
                else:
                    batch_size = data.size(0)
                    n_iter = len(train_loader) * epoch
                    vv[idx*batch_size:(idx+1)*batch_size, :] = model.encode(data.view(-1, conf.n_vis).to(conf.device)).detach().cpu().numpy()
            for idx, (data, target) in enumerate(train_loader):
                if conf.whitening_learn:
                    batch_size = data.size(0)
                    n_step = len(train_loader) * epoch + idx
                    batch_white = np.dot(data.view(batch_size, -1).numpy(), P)
                    batch_white = torch.from_numpy(batch_white.astype(np.float32)).clone().view(batch_size, -1)
                    loss = model.train_step(batch_white.to(conf.device)).cpu()
                    writer.add_scalar(f"{model.__class__.__name__}/{str(conf)}/loss", loss, n_step)
                    recon_loss = model.recon_loss(batch_white.to(device)).cpu()
                    writer.add_scalar(f"{model.__class__.__name__}/{str(conf)}/recon_loss", recon_loss, n_step)  


                else:
                    batch_size = data.size(0)
                    n_step = len(train_loader) * epoch + idx
                    loss = model.train_step(data.to(device)).cpu()
                    writer.add_scalar(f"{model.__class__.__name__}/{str(conf)}/loss", loss, n_step)
                    recon_loss = model.recon_loss(data.to(device)).cpu()
                    writer.add_scalar(f"{model.__class__.__name__}/{str(conf)}/recon_loss", recon_loss, n_step)                
        yield epoch, n_iter, vv
    
model = AutoEncoder(conf).to(device)
vv_iter = generate_vv(model, conf)

In [4]:
import matplotlib
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

if not conf.whitening_learn:
    sample_images = next(iter(test_loader))[0]
else:
    sample_images = next(iter(test_loader))[0]
    sample_images = np.dot(sample_images.numpy().reshape(-1, conf.n_vis), P)
    sample_images = torch.from_numpy(sample_images.astype(np.float32)).clone()


sample_img = make_grid(sample_images.view(conf.batch_size, 1, 32, 32).data)

fig, ax = plt.subplots(figsize=(8, 8))
divider = make_axes_locatable(ax)
mappable = cm.ScalarMappable(cmap=cm.viridis, norm=matplotlib.colors.Normalize(vmin=0, vmax=conf.n_epoch))
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(mappable=mappable,cax=cax, label="Epoch")


xx = np.arange(conf.n_hid) + 1
log_xx = np.log(xx)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel('Neurons')
ax.set_ylabel('Variance explained')

pca = PCA()
pca.fit(train_datasets.data.reshape(-1, conf.n_vis))
yy = pca.explained_variance_
log_yy = np.log(yy)
m, c = np.linalg.lstsq(np.vstack([log_xx, np.ones(len(log_xx))]).T, log_yy, rcond=None)[0]
ax.plot(xx, yy, label="Input dataset")
ax.legend()
for n_epoch, n_iter, vv in vv_iter:

    pca = PCA()
    pca.fit(vv)
    yy = pca.explained_variance_
    log_yy = np.log(yy)
    m, c = np.linalg.lstsq(np.vstack([log_xx, np.ones(len(log_xx))]).T, log_yy, rcond=None)[0]
    pcm = ax.plot(xx, yy, '.-', color=cm.viridis(n_epoch / conf.n_epoch))
    # ax.legend()
    ax.grid(visible=True)
    writer.add_figure(f"{model.__class__.__name__}/{str(conf)}/neuron_fireings", fig, global_step=n_iter)

    v_recon = model(sample_images.view(conf.batch_size, -1).to(device))
    sample_recon = make_grid(v_recon.view(conf.batch_size, 3, 32, 32).data).cpu()
    writer.add_image(f"{model.__class__.__name__}/{str(conf)}/sample_input", sample_img, global_step=n_iter)
    writer.add_image(f"{model.__class__.__name__}/{str(conf)}/sample_recon", sample_recon, global_step=n_iter)
#ax.legend()
#fig.colorbar(ax)
ax.grid(visible=True)
writer.add_figure(f"{model.__class__.__name__}/{str(conf)}/neuron_fireings", fig)

RuntimeError: shape '[128, 3, 32, 32]' is invalid for input of size 131072